In [1]:
%%spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1601463335080_0107,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import *
import pandas as pd
import matplotlib.pyplot as plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
output_data = "s3a://matrix-data-lake-output/"

songs = spark.read.parquet(output_data+"/songs_table")
songs.createOrReplaceTempView("songs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
songs.groupBy("year","artist_id").agg(count("*").alias("counter")).orderBy(desc("counter")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------+-------+
|year|         artist_id|counter|
+----+------------------+-------+
|   0|ARR3ONV1187B9A2F59|      7|
|   0|ARSF0K11187B9AF319|      6|
|   0|AROF4LP1187FB41C51|      6|
|   0|AR33DV81187FB4139B|      6|
|   0|ARWV4871187B98BF87|      5|
|   0|ARFULGF1187FB5690C|      5|
|   0|ARBW1J61187B9B8B54|      5|
|   0|ARU1MVY1187FB37B73|      5|
|   0|ARQT8QM1187FB3E3CB|      5|
|   0|ARE2G8K1187FB53219|      5|
|   0|ARY5UO61187FB5271F|      5|
|   0|ARXOTQH1187FB57084|      5|
|   0|ARKJBR01187FB429D7|      5|
|   0|ARFWGRI1187FB52E45|      5|
|   0|AR32JLC1187B9946AF|      5|
|   0|AR7KA5V1187FB44E6B|      5|
|   0|ARCGQQ11187B994F5D|      4|
|   0|ARJD43U1187B98EAB9|      4|
|   0|AR1ZKBE1187FB53629|      4|
|   0|ARLHO5Z1187FB4C861|      4|
+----+------------------+-------+
only showing top 20 rows

In [20]:
songsTablePartitions = songs.select("year","artist_id").dropDuplicates()
songsTablePartitions.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12620

In [14]:
songplays = spark.read.parquet(output_data+"/songplays_table")
songplays.createOrReplaceTempView("songplays")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Top 3 Songs Played

In [15]:
top3PlayedSongs = songplays.join(songs,songplays.song_id == songs.song_id,"left")\
                          .select(songplays.song_id,songs.title).where(songplays.song_id.isNotNull())
top3PlayedSongs = top3PlayedSongs.groupBy(col("song_id"),col("title")).count().orderBy(desc("count")).limit(3)
top3PlayedSongs.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------+-----+
|           song_id|         title|count|
+------------------+--------------+-----+
|SOBONKR12A58A7A7E0|You're The One|   37|
|SOTTKFE12A6D4F9AFE|       Secrets|   17|
|SORKXUL12AB01821DA|          Home|   13|
+------------------+--------------+-----+

## Top 3 Artists

In [16]:
artists = spark.read.parquet(output_data+"/artists_table")
artists.createOrReplaceTempView("artists")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
spark.sql("""
                SELECT t2.name, count(*) 
                FROM songplays AS t1 
                JOIN artists AS t2 ON t1.artist_id = t2.artist_id 
                GROUP BY t2.name 
                ORDER BY count(*) DESC 
                LIMIT 3
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------+
|            name|count(1)|
+----------------+--------+
|   Dwight Yoakam|      37|
|Carleen Anderson|      17|
|   Frozen Plasma|      13|
+----------------+--------+

## Top 3 hours listening activity

In [18]:
time = spark.read.parquet(output_data+"/time_table")
time.createOrReplaceTempView("time")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
spark.sql("""
            SELECT t2.hour, count(*) 
            FROM songplays AS t1 
            JOIN time AS t2 ON t1.start_time = t2.start_time 
            GROUP BY t2.hour 
            ORDER BY count(*) DESC 
            LIMIT 3
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|hour|count(1)|
+----+--------+
|  16|     560|
|  18|     527|
|  17|     509|
+----+--------+